In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
import os

In [2]:
img_size = 784
hidden_dim = 400
latent_dim = 20
batch_size = 128
epochs = 10

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dataset = torchvision.datasets.MNIST(root="./data",
                                                  train = True,
                                                  transform = transforms.ToTensor(),
                                                  download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_dataset = torchvision.datasets.MNIST(root="./data",
                                                  train = False,
                                                  transform = transforms.ToTensor(),
                                                  download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

# save img to dir
sample_dir = "output"

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

100%|██████████| 9912422/9912422 [00:00<00:00, 100108621.70it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 105243869.53it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30092250.98it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 988200.48it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Model

class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()

        self.fc1 = nn.Linear(img_size,hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim,latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim,latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, img_size)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)

        return mu, logvar

    def reparameterize(self, mu,logvar):
        std = torch.exp(logvar/2)
        eps = torch.randn_like(std)

        return mu + eps*std

    def decode(self,z):
        h = F.relu(self.fc3(z))
        out = torch.sigmoid(self.fc4(h))

        return out

    def forward(self, x):
        mu,logvar = self.encode(x.view(-1,img_size))
        z = self.reparameterize(mu,logvar)
        out = self.decode(z)

        return out,mu,logvar


model = VAE().to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def loss_fn(out,img,mu,logvar):
    bce = F.binary_cross_entropy(out,img.view(-1,img_size), reduction="sum")
    kld = 0.5*torch.sum(logvar.exp() + mu**2 - 1 - logvar)

    return bce+kld

def train(epoch):
    model.train()
    train_loss = 0

    for i,(images,_) in enumerate(train_loader):
        images = images.to(device)
        out,mu,logvar = model(images)
        loss = loss_fn(out,images,mu,logvar)
        optim.zero_grad()
        loss.backward()
        optim.step()

        train_loss+=loss.item()

        if i%100==0:
            print("Train Epoch {} [Batch {}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), loss.item()/len(images)))

    print(f"Epoch {epoch}, Average Loss: {(train_loss/len(train_loader.dataset)):.3f}")

def test(epoch):
    model.eval()

    test_loss = 0

    with torch.no_grad():
        for batch_idx,(images,_) in enumerate(test_loader):
            images = images.to(device)
            out,mu,logvar = model(images)
            test_loss += loss_fn(out,images,mu,logvar).item()

            if batch_idx == 0:
                comparison = torch.cat([images[:5], out.view(batch_size,1,28,28)[:5]])
                save_image(comparison.cpu(), 'output/out_' + str(epoch) + '.png',nrow=5)

        print(f">> Average Loss: {test_loss/len(test_loader.dataset):.3f}")

In [7]:
for epoch in range(1, epochs+1):
    train(epoch)
    test(epoch)

    with torch.no_grad():
        sample = torch.randn(64,20).to(device)
        generated = model.decode(sample).cpu()
        save_image(generated.view(64,1,28,28), 'output/sample_' + str(epoch)+'.png')

Train Epoch 1 [Batch 0/469]	Loss: 549.777
Train Epoch 1 [Batch 100/469]	Loss: 183.312
Train Epoch 1 [Batch 200/469]	Loss: 154.053
Train Epoch 1 [Batch 300/469]	Loss: 139.956
Train Epoch 1 [Batch 400/469]	Loss: 130.133
Epoch 1, Average Loss: 164.566
>> Average Loss: 127.882
Train Epoch 2 [Batch 0/469]	Loss: 129.915
Train Epoch 2 [Batch 100/469]	Loss: 123.803
Train Epoch 2 [Batch 200/469]	Loss: 128.375
Train Epoch 2 [Batch 300/469]	Loss: 118.118
Train Epoch 2 [Batch 400/469]	Loss: 115.804
Epoch 2, Average Loss: 121.966
>> Average Loss: 116.304
Train Epoch 3 [Batch 0/469]	Loss: 119.898
Train Epoch 3 [Batch 100/469]	Loss: 118.642
Train Epoch 3 [Batch 200/469]	Loss: 112.803
Train Epoch 3 [Batch 300/469]	Loss: 118.791
Train Epoch 3 [Batch 400/469]	Loss: 109.729
Epoch 3, Average Loss: 114.898
>> Average Loss: 112.291
Train Epoch 4 [Batch 0/469]	Loss: 110.326
Train Epoch 4 [Batch 100/469]	Loss: 115.354
Train Epoch 4 [Batch 200/469]	Loss: 115.141
Train Epoch 4 [Batch 300/469]	Loss: 109.236
Trai